Data from https://healthdata.gov/Health/COVID-19-Public-Therapeutic-Locator/rxn6-qnx8

In [1]:
#import library
import pandas as pd
import numpy as np
import folium
import matplotlib

### Metadata

DataframeA will include the older stocking data while DataframeB is the latest stock data

In [2]:
#define data to input

#A is the newest month, B will be the older month
df_A = pd.read_csv('C:/Users/Sydney/Desktop/apa_stock_data/COVID-19_Public_Therapeutic_Locator_20230201.csv')
df_B = pd.read_csv('C:/Users/Sydney/Desktop/apa_stock_data/COVID-19_Public_Therapeutic_Locator_20220729.csv')

date_A = 20230201 #this is the latest date of the report download
date_B = 20220729

therapy = "Paxlovid"

### Data Exploration

In [3]:
df_A.head()
df_A.columns

Index(['Provider Name', 'Address1', 'Address2', 'City', 'County', 'State Code',
       'Zip', 'National Drug Code', 'Order Label', 'Courses Available',
       'Geocoded Address', 'NPI', 'Last Report Date', 'Provider Status',
       'Provider Note'],
      dtype='object')

In [4]:
#what does our provider status tell us
df_A['Provider Status'].value_counts()

ACTIVE               98982
UNKNOWN INVENTORY     4246
Name: Provider Status, dtype: int64

In [5]:
#what are the unique order lables - anything we don't care about?
df_A["Order Label"].unique()

array(['Paxlovid', 'Renal Paxlovid', 'Lagevrio (molnupiravir)',
       'Commercial Veklury (remdesivir)'], dtype=object)

### Clean and prepare the data

In [6]:
def clean_df(df, date):
    
    #drop all unnecessary columns for this report and limit to only where the provider status is active
    df = df[df["Provider Status"] == "ACTIVE"].drop(labels = ['Provider Name', 'Address1', 'Address2','City',
                                                                'National Drug Code','NPI','Provider Note'], axis = 1)
    
    #clena up the geocoded address to be the coordinates only and drop any NA values in that column
    df = df.dropna(subset=['Geocoded Address'])
    df["Geocoded Address"] = df["Geocoded Address"].astype("string").str.strip('POINT (').str.strip(')')

    split_data = df["Geocoded Address"].str.split(' ')
    df['long'] = split_data.apply(lambda x: x[0])
    df['lat'] = split_data.apply(lambda x: x[1])
    df.head()
    
    new_col = "Courses Available" + str(date)
    df = df.rename(columns={"Courses Available": new_col})
    
    return df

In [7]:
df_A = clean_df(df_A, date_A)
df_B = clean_df(df_B, date_B)

In [8]:
df_A.head()

,County,State Code,Zip,Order Label,Courses Available20230201,Geocoded Address,Last Report Date,Provider Status,long,lat
0,Adams,CO,80233,Paxlovid,55.0,-104.955542 39.91345,01/25/2023 12:00:00 AM,ACTIVE,-104.955542,39.91345
1,Faulkner,AR,72032,Paxlovid,35.0,-92.437225 35.09223,01/30/2023 12:00:00 AM,ACTIVE,-92.437225,35.09223
2,Cook,IL,60302,Paxlovid,6.0,-87.79498 41.8795,01/30/2023 12:00:00 AM,ACTIVE,-87.79498,41.8795
3,Sullivan,MO,63556,Paxlovid,9.0,-93.12537 40.202935,01/19/2023 12:00:00 AM,ACTIVE,-93.12537,40.202935
4,Brown,WI,54303,Paxlovid,36.0,-88.047111 44.51736,01/30/2023 12:00:00 AM,ACTIVE,-88.047111,44.51736


## All Therapies

### Find the Differences in Stock Over Time

In [9]:
def zip_change(df1, df2, date1, date2):
    df1 = df1.drop(labels = ['State Code', 'County', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["Zip","Order Label"],
                                                                                 as_index = False).sum()
    df2 = df2.drop(labels = ['State Code', 'County', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["Zip","Order Label"],
                                                                                 as_index = False).sum()
    
    df3 = pd.merge(df1,df2,on=["Zip", "Order Label"])
    
    df3['Delta'] = df3['Courses Available'+str(date1)] - df3['Courses Available'+str(date2)]
    
    return df3

In [10]:
zip_df = zip_change(df_A, df_B, date_A, date_B)

In [11]:
def cnty_change(df1, df2, date1, date2):
    df1 = df1.drop(labels = ['State Code', 'Zip', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["County","Order Label"],
                                                                                 as_index = False).sum()
    df2 = df2.drop(labels = ['State Code', 'Zip', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["County","Order Label"],
                                                                                 as_index = False).sum()
    
    df3 = pd.merge(df1,df2,on=["County", "Order Label"])
    
    df3['Delta'] = df3['Courses Available'+str(date1)] - df3['Courses Available'+str(date2)]
    
    return df3

In [12]:
cnty_df = cnty_change(df_A, df_B, date_A, date_B)

In [13]:
def state_change(df1, df2, date1, date2):
    df1 = df1.drop(labels = ['Zip', 'County', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["State Code","Order Label"],
                                                                                 as_index = False).sum()
    df2 = df2.drop(labels = ['Zip', 'County', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["State Code","Order Label"],
                                                                                  as_index = False).sum()
    
    df3 = pd.merge(df1,df2,on=["State Code", "Order Label"])
    
    df3['Delta'] = df3['Courses Available'+str(date1)] - df3['Courses Available'+str(date2)]
    
    return df3

In [14]:
state_df = state_change(df_A, df_B, date_A, date_B)

In [15]:
def nat_change(df1, df2, date1, date2):
    df1 = df1.drop(labels = ['State Code', 'County', 'Zip', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["Order Label"],
                                                                                 as_index = False).sum()
    df2 = df2.drop(labels = ['State Code', 'County', 'Zip', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["Order Label"],
                                                                                 as_index = False).sum()
    
    df3 = pd.merge(df1,df2,on=["Order Label"])
    
    df3['Delta'] = df3['Courses Available'+str(date1)] - df3['Courses Available'+str(date2)]
    
    return df3

In [16]:
nat_df = nat_change(df_A, df_B, date_A, date_B)

In [17]:
nat_df.head()

,Order Label,Courses Available20230201,Courses Available20220729,Delta
0,Lagevrio (molnupiravir),1126283.0,1449110.0,-322827.0
1,Paxlovid,1566066.0,1237893.0,328173.0
2,Renal Paxlovid,233780.0,187670.0,46110.0


### Export the data summaries

In [18]:
nat_df.to_excel(r'C:\Users\Sydney\Desktop\apa_stock_data\APA_stock_change_National.xlsx', index=True)
state_df.to_excel(r'C:\Users\Sydney\Desktop\apa_stock_data\APA_stock_change_State.xlsx', index=True)
zip_df.to_excel(r'C:\Users\Sydney\Desktop\apa_stock_data\APA_stock_change_Zip.xlsx', index=True)
cnty_df.to_excel(r'C:\Users\Sydney\Desktop\apa_stock_data\APA_stock_change_Cnty.xlsx', index=True)

## Select a Single Therapy

### Find the Differences in Stock Over Time 

In [19]:
df_A2 = df_A[df_A["Order Label"] == therapy]
df_B2 = df_B[df_B["Order Label"] == therapy]

In [20]:
def zip_change_single(df1, df2, date1, date2):
    df1 = df1.drop(labels = ['State Code', 'County', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status', 'Order Label'], axis = 1).groupby(["Zip"],
                                                                                                as_index = False).sum()
    df2 = df2.drop(labels = ['State Code', 'County', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status', 'Order Label'], axis = 1).groupby(["Zip"],
                                                                                                as_index = False).sum()
    
    df3 = pd.merge(df1,df2,on=["Zip"])
    
    df3['Delta'] = df3['Courses Available'+str(date1)] - df3['Courses Available'+str(date2)]
    
    return df3

In [21]:
zip_df2 = zip_change_single(df_A2, df_B2, date_A, date_B)

In [22]:
def cnty_change_single(df1, df2, date1, date2):
    df1 = df1.drop(labels = ['State Code', 'Zip', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status', 'Order Label'], axis = 1).groupby(["County"],
                                                                                                as_index = False).sum()
    df2 = df2.drop(labels = ['State Code', 'Zip', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status', 'Order Label'], axis = 1).groupby(["County"],
                                                                                                as_index = False).sum()
    
    df3 = pd.merge(df1,df2,on=["County"])
    
    df3['Delta'] = df3['Courses Available'+str(date1)] - df3['Courses Available'+str(date2)]
    
    return df3

In [23]:
cnty_df2 = cnty_change_single(df_A2, df_B2, date_A, date_B)

In [24]:
def state_change_single(df1, df2, date1, date2):
    df1 = df1.drop(labels = ['Zip', 'County', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status', 'Order Label'], axis = 1).groupby(["State Code"],
                                                                                                as_index = False).sum()
    df2 = df2.drop(labels = ['Zip', 'County', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status', 'Order Label'], axis = 1).groupby(["State Code"],
                                                                                                as_index = False).sum()
    
    df3 = pd.merge(df1,df2,on=["State Code"])
    
    df3['Delta'] = df3['Courses Available'+str(date1)] - df3['Courses Available'+str(date2)]
    
    return df3

In [25]:
state_df2 = state_change_single(df_A2, df_B2, date_A, date_B)

In [26]:
state_df2.head()

,State Code,Courses Available20230201,Courses Available20220729,Delta
0,AK,2038.0,2068.0,-30.0
1,AL,17872.0,16468.0,1404.0
2,AR,9382.0,8995.0,387.0
3,AS,1657.0,1594.0,63.0
4,AZ,26425.0,23148.0,3277.0


In [27]:
def nat_change_single(df1, df2, date1, date2):
    df1 = df1.drop(labels = ['State Code', 'County', 'Zip', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["Order Label"],
                                                                                 as_index = False).sum()
    df2 = df2.drop(labels = ['State Code', 'County', 'Zip', 'Geocoded Address','Last Report Date', 
                             'long', 'lat', 'Provider Status'], axis = 1).groupby(["Order Label"],
                                                                                 as_index = False).sum()
    
    df3 = pd.merge(df1,df2,on=["Order Label"])
    
    df3['Delta'] = df3['Courses Available'+str(date1)] - df3['Courses Available'+str(date2)]
    
    return df3

In [28]:
nat_df2 = nat_change_single(df_A2, df_B2, date_A, date_B)

In [29]:
nat_df2.head()

,Order Label,Courses Available20230201,Courses Available20220729,Delta
0,Paxlovid,1566066.0,1237893.0,328173.0


### Export the data summaries

In [30]:
nat_df2.to_excel(r'C:\Users\Sydney\Desktop\apa_stock_data\Single_stock_change_National.xlsx', index=True)
state_df2.to_excel(r'C:\Users\Sydney\Desktop\apa_stock_data\Single_stock_change_State.xlsx', index=True)
zip_df2.to_excel(r'C:\Users\Sydney\Desktop\apa_stock_data\Single_stock_change_Zip.xlsx', index=True)
cnty_df2.to_excel(r'C:\Users\Sydney\Desktop\apa_stock_data\Single_stock_change_Cnty.xlsx', index=True)

### Quick Summaries

In [31]:
#States with the biggest drop in supply
state_df2.sort_values(by="Delta").head(10)

,State Code,Courses Available20230201,Courses Available20220729,Delta
25,MN,16504.0,23671.0,-7167.0
21,MA,47858.0,50734.0,-2876.0
36,NV,9579.0,10917.0,-1338.0
23,ME,8946.0,10265.0,-1319.0
20,LA,13949.0,15082.0,-1133.0
49,UT,7695.0,8497.0,-802.0
13,HI,4971.0,5678.0,-707.0
14,IA,8188.0,8810.0,-622.0
11,FM,2896.0,3470.0,-574.0
46,SD,1125.0,1616.0,-491.0


In [32]:
#States with the largest gain in supply
state_df2.sort_values(by="Delta").tail(10)

,State Code,Courses Available20230201,Courses Available20220729,Delta
22,MD,37035.0,26908.0,10127.0
24,MI,42743.0,31760.0,10983.0
16,IL,63768.0,52493.0,11275.0
38,OH,45254.0,33127.0,12127.0
34,NJ,60365.0,44099.0,16266.0
37,NY,90026.0,70109.0,19917.0
48,TX,133570.0,106695.0,26875.0
6,CO,61349.0,19402.0,41947.0
41,PA,87032.0,40833.0,46199.0
5,CA,227638.0,162710.0,64928.0


In [33]:
#Counties with the greatest loss of supply
cnty_df2.sort_values(by="Delta").head(15)

,County,Courses Available20230201,Courses Available20220729,Delta
1007,Norfolk,13653.0,21056.0,-7403.0
189,Broward,8274.0,10708.0,-2434.0
930,Milwaukee,4565.0,6579.0,-2014.0
601,Hanover,763.0,2529.0,-1766.0
1157,Ramsey,1331.0,3045.0,-1714.0
518,Fresno,4349.0,5770.0,-1421.0
1086,Penobscot,1274.0,2614.0,-1340.0
433,Dougherty,1505.0,2728.0,-1223.0
1120,Polk,4955.0,6147.0,-1192.0
201,Buncombe,1557.0,2689.0,-1132.0


In [34]:
#Counties with the greatest gain of supply
cnty_df2.sort_values(by="Delta").tail(15)

,County,Courses Available20230201,Courses Available20220729,Delta
1242,San Bernardino,9085.0,5615.0,3470.0
741,Kern,7593.0,3969.0,3624.0
923,Middlesex,15998.0,11802.0,4196.0
1153,Queens,11251.0,6943.0,4308.0
746,King,12265.0,7905.0,4360.0
352,Cook,28513.0,22938.0,5575.0
1243,San Diego,17217.0,10771.0,6446.0
1183,Riverside,18252.0,10539.0,7713.0
1040,Orange,26178.0,18252.0,7926.0
16,Alameda,13980.0,6019.0,7961.0


## Maps